# Epic 5:

Als een key user kan ik voor een campagne een lijst met contacten genereren volgens de waarschijnlijkheid om in te schrijven voor de campagne. Ik wil in de sortering ook ervoor zorgen dat contacten met weinig marketing pressure bevoordeeld worden ten opzichte van contacten met een hoge marketing pressure. 

In [1]:
# Importing the necessary packages
import numpy as np                                  # "Scientific computing"
import scipy.stats as stats                         # Statistical tests

import pandas as pd                                 # Data Frame
from pandas.api.types import CategoricalDtype

import matplotlib.pyplot as plt                     # Basic visualisation
from statsmodels.graphics.mosaicplot import mosaic  # Mosaic diagram
import seaborn as sns   

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity

from dotenv import load_dotenv
import os
from sqlalchemy import create_engine, MetaData, select
import pandas as pd


## Database connection

In [2]:
load_dotenv()
csv_dir =  '/home/flor/Workspace/DEP2/DataEngineerProj2/Data'
SERVER = os.environ.get('SERVER')
DATABASE = os.environ.get('DATAWAREHOUSE')
UID = os.environ.get('USER')
PWD = os.environ.get('PASSWORD')

connection_string = f'mssql+pyodbc://{UID}:{PWD}@{SERVER}/{DATABASE}?driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(connection_string)

metadata = MetaData()
metadata.reflect(engine)

### Creating DataFrame

In [3]:
df_fact = pd.read_sql_table(table_name='FactCampagne', con=engine)
df_mail = pd.read_sql_table(table_name='DimEmail', con=engine)
df_cont = pd.read_sql_table(table_name='DimContact', con=engine)
df_cust = pd.read_sql_table(table_name='DimCustomer', con=engine)
df_final = pd.merge(df_fact, df_mail, on="Mailing_ID", how="left")

df_final.drop(["Visit_ID_y",
               "Campagne_ID_y",
               "Contact_ID_y"], axis=1, inplace=True)
df_final.rename(columns={
    'Contact_ID_x': 'Contact_ID',
    'Visit_ID_x': 'Visit_ID',
    'Campagne_ID_x': 'Campagne_ID'
}, inplace=True)

df_cust.drop(["Postcode", "Industriezone_Naam_", "Is_Voka_entiteit", 'Oprichtingsdatum', 
              'Primaire_activiteit', 'Reden_van_status', 'Status', 'Voka_Nr_', "Persoon_ID", "Persoonnr", "Status_Persoon",
              "Lidmaatschap_ID",'Opzeg', "Reden_Aangroei", "Reden_Verloop", "Start_Datum"], axis=1, inplace=True)


# df_final = pd.merge(df_final, df_cust, how="left")

df_final = pd.merge(df_final, df_cont, on="Contact_ID", how="left")

df_final.drop(['Persoon_ID_y', 'Inschrijving_ID_y'], axis=1, inplace=True)

df_final.rename(columns={
    'Persoon_ID_x': 'Persoon_ID',
    'Inschrijving_ID_x': 'Inschrijving_ID',
}, inplace=True)

df_final.head()

,Customer_ID,Persoon_ID,Contact_ID,Inschrijving_ID,Campagne_ID,Mailing_ID,Visit_ID,Campagne_Nr,Eind_date_key,Naam_Campagne,...,IP_Adress,IP_Organization,Account_ID,Functie_title,Contact_status,Voka_medewerker,Inschrijving_status,Bron,Datum,Facturatie_bedrag
0,E40F33E0-C268-E111-B43A-00505680000A,59586FD2-A913-E211-9DAA-005056B06EB4,90122280-58FD-E811-80F9-001DD8B72B61,1117EF05-AEE2-ED11-A7C7-6045BD895403,A2F63005-8EA8-ED11-AAD1-6045BD895D85,AD97B2C6-D6D9-ED11-A7C7-6045BD89520A,5DAB6B2C-9507-459C-8049-6B2BED9F7D78,23-OV-01-0056,20230525,OV-NW-Voka Bilan 2023,...,87.65.231.115,477664A5-4170-ED11-9561-6045BD8952CE,E40F33E0-C268-E111-B43A-00505680000A,Algemeen Directeur - CEO,Actief,0,Aanwezig,Website,2023-04-24,0
1,E40F33E0-C268-E111-B43A-00505680000A,59586FD2-A913-E211-9DAA-005056B06EB4,90122280-58FD-E811-80F9-001DD8B72B61,1117EF05-AEE2-ED11-A7C7-6045BD895403,A2F63005-8EA8-ED11-AAD1-6045BD895D85,AD97B2C6-D6D9-ED11-A7C7-6045BD89520A,5DAB6B2C-9507-459C-8049-6B2BED9F7D78,23-OV-01-0056,20230525,OV-NW-Voka Bilan 2023,...,87.65.231.115,477664A5-4170-ED11-9561-6045BD8952CE,E40F33E0-C268-E111-B43A-00505680000A,Algemeen Directeur - CEO,Actief,0,Aanwezig,Email,2023-09-28,0
2,E40F33E0-C268-E111-B43A-00505680000A,59586FD2-A913-E211-9DAA-005056B06EB4,90122280-58FD-E811-80F9-001DD8B72B61,1117EF05-AEE2-ED11-A7C7-6045BD895403,A2F63005-8EA8-ED11-AAD1-6045BD895D85,AD97B2C6-D6D9-ED11-A7C7-6045BD89520A,5DAB6B2C-9507-459C-8049-6B2BED9F7D78,23-OV-01-0056,20230525,OV-NW-Voka Bilan 2023,...,87.65.231.115,477664A5-4170-ED11-9561-6045BD8952CE,E40F33E0-C268-E111-B43A-00505680000A,Algemeen Directeur - CEO,Actief,0,Aanwezig,Website,2023-10-09,0
3,E40F33E0-C268-E111-B43A-00505680000A,59586FD2-A913-E211-9DAA-005056B06EB4,90122280-58FD-E811-80F9-001DD8B72B61,1117EF05-AEE2-ED11-A7C7-6045BD895403,A2F63005-8EA8-ED11-AAD1-6045BD895D85,AD97B2C6-D6D9-ED11-A7C7-6045BD89520A,5DAB6B2C-9507-459C-8049-6B2BED9F7D78,23-OV-01-0056,20230525,OV-NW-Voka Bilan 2023,...,87.65.231.115,477664A5-4170-ED11-9561-6045BD8952CE,E40F33E0-C268-E111-B43A-00505680000A,Algemeen Directeur - CEO,Actief,0,Aanwezig,Email,2022-12-15,70
4,E40F33E0-C268-E111-B43A-00505680000A,59586FD2-A913-E211-9DAA-005056B06EB4,90122280-58FD-E811-80F9-001DD8B72B61,1117EF05-AEE2-ED11-A7C7-6045BD895403,A2F63005-8EA8-ED11-AAD1-6045BD895D85,AD97B2C6-D6D9-ED11-A7C7-6045BD89520A,5DAB6B2C-9507-459C-8049-6B2BED9F7D78,23-OV-01-0056,20230525,OV-NW-Voka Bilan 2023,...,87.65.231.115,477664A5-4170-ED11-9561-6045BD8952CE,E40F33E0-C268-E111-B43A-00505680000A,Algemeen Directeur - CEO,Actief,0,Aanwezig,Email,2023-01-10,0


In [4]:
df_final["Inschrijving_status"].value_counts()

Inschrijving_status
Aanwezig    15315538
Afwezig       378758
Name: count, dtype: int64

In [5]:
id_columns = df_final[['Customer_ID', 'Persoon_ID', 'Contact_ID', 'Inschrijving_ID', 'Campagne_ID', 'Mailing_ID', 'Visit_ID', 'Account_ID']]
df_final_strip = df_final.drop(id_columns, axis=1)

# IP ORGANIZATION, MAILING NAME
df_final_strip.drop(['IP_Organization', 'Mailing_Name', 'Mailing_Sent_On', 'Datum', 'IP_Adress'], axis=1, inplace=True)

# df_final_strip['Mailing_Sent_On'] = pd.to_datetime(df_final_strip['Mailing_Sent_On'])
# df_final_strip['Datum'] = pd.to_datetime(df_final_strip['Datum'])

df_final_strip.head()

,Campagne_Nr,Eind_date_key,Naam_Campagne,Naam_in_email,Reden_van_status,Start_date_key,Status_Camp,Type_campagne,Soort_Campagne,Onderwerp_campagne,Mailing_Subject,IP_Stad,IP_Land,Functie_title,Contact_status,Voka_medewerker,Inschrijving_status,Bron,Facturatie_bedrag
0,23-OV-01-0056,20230525,OV-NW-Voka Bilan 2023,Voka's statutaire jaarvergadering & jaarfeest ...,Voltooid,20230525,Actief,Netwerkevenement,Offline,NW,Uitnodiging: Voka Jaarfeest 2023,Ghent,Belgium,Algemeen Directeur - CEO,Actief,0,Aanwezig,Website,0
1,23-OV-01-0056,20230525,OV-NW-Voka Bilan 2023,Voka's statutaire jaarvergadering & jaarfeest ...,Voltooid,20230525,Actief,Netwerkevenement,Offline,NW,Uitnodiging: Voka Jaarfeest 2023,Ghent,Belgium,Algemeen Directeur - CEO,Actief,0,Aanwezig,Email,0
2,23-OV-01-0056,20230525,OV-NW-Voka Bilan 2023,Voka's statutaire jaarvergadering & jaarfeest ...,Voltooid,20230525,Actief,Netwerkevenement,Offline,NW,Uitnodiging: Voka Jaarfeest 2023,Ghent,Belgium,Algemeen Directeur - CEO,Actief,0,Aanwezig,Website,0
3,23-OV-01-0056,20230525,OV-NW-Voka Bilan 2023,Voka's statutaire jaarvergadering & jaarfeest ...,Voltooid,20230525,Actief,Netwerkevenement,Offline,NW,Uitnodiging: Voka Jaarfeest 2023,Ghent,Belgium,Algemeen Directeur - CEO,Actief,0,Aanwezig,Email,70
4,23-OV-01-0056,20230525,OV-NW-Voka Bilan 2023,Voka's statutaire jaarvergadering & jaarfeest ...,Voltooid,20230525,Actief,Netwerkevenement,Offline,NW,Uitnodiging: Voka Jaarfeest 2023,Ghent,Belgium,Algemeen Directeur - CEO,Actief,0,Aanwezig,Email,0


In [6]:
# unsure about these parameters
# Reden van status, Contact_status, Voka_medewerker, Bron
df_final_strip.drop(['Naam_Campagne', 'Naam_in_email', 'Mailing_Subject', "Reden_van_status", "Voka_medewerker", "Eind_date_key", "Start_date_key", 'Campagne_Nr'], axis=1, inplace=True)
df_final_strip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15694296 entries, 0 to 15694295
Data columns (total 11 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   Status_Camp          object
 1   Type_campagne        object
 2   Soort_Campagne       object
 3   Onderwerp_campagne   object
 4   IP_Stad              object
 5   IP_Land              object
 6   Functie_title        object
 7   Contact_status       object
 8   Inschrijving_status  object
 9   Bron                 object
 10  Facturatie_bedrag    object
dtypes: object(11)
memory usage: 1.3+ GB


In [7]:
df_final_strip.head()

,Status_Camp,Type_campagne,Soort_Campagne,Onderwerp_campagne,IP_Stad,IP_Land,Functie_title,Contact_status,Inschrijving_status,Bron,Facturatie_bedrag
0,Actief,Netwerkevenement,Offline,NW,Ghent,Belgium,Algemeen Directeur - CEO,Actief,Aanwezig,Website,0
1,Actief,Netwerkevenement,Offline,NW,Ghent,Belgium,Algemeen Directeur - CEO,Actief,Aanwezig,Email,0
2,Actief,Netwerkevenement,Offline,NW,Ghent,Belgium,Algemeen Directeur - CEO,Actief,Aanwezig,Website,0
3,Actief,Netwerkevenement,Offline,NW,Ghent,Belgium,Algemeen Directeur - CEO,Actief,Aanwezig,Email,70
4,Actief,Netwerkevenement,Offline,NW,Ghent,Belgium,Algemeen Directeur - CEO,Actief,Aanwezig,Email,0


In [8]:
df_final_strip[['Status_Camp', 'Type_campagne', 'Soort_Campagne', 'IP_Stad', 'IP_Land',
      'Inschrijving_status',  'Onderwerp_campagne', 'Bron', 'Functie_title']] = df_final_strip[
  ['Status_Camp', 'Type_campagne', 'Soort_Campagne', 'IP_Stad', 'IP_Land',
      'Inschrijving_status',  'Onderwerp_campagne', 'Bron', 'Functie_title']].astype(str)

In [9]:
df_final_strip["Facturatie_bedrag"] = pd.to_numeric(df_final_strip["Facturatie_bedrag"], errors="coerce")

# df_final_strip["Facturatie_bedrag"] = df_final_strip["Facturatie_bedrag"].astype("float64")
# df_final_strip['Facturatie_bedrag'].value_counts()

## Data Preprocessing


### Handling Text and Categorical Attributes

In [10]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

cat_attribs = ['Status_Camp', 'Type_campagne', 'Soort_Campagne', 'IP_Stad', 'IP_Land',
      'Inschrijving_status',  'Onderwerp_campagne', 'Bron', 'Functie_title']

num_attribs = ['Facturatie_bedrag']


cat_pipeline = Pipeline([
    ("encoder", OrdinalEncoder()),
    ('imputer', SimpleImputer())
])

num_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ('imputer', SimpleImputer(strategy="mean"))
])

prep = ColumnTransformer([
    ('cat', cat_pipeline, cat_attribs),
    ('num', num_pipeline, num_attribs)
])

df_final_strip = prep.fit_transform(df_final_strip)

In [11]:
from sklearn.model_selection import train_test_split

df_final_strip = pd.DataFrame(df_final_strip, columns=cat_attribs + num_attribs)


X = df_final_strip.drop('Inschrijving_status', axis=1)
y = df_final_strip['Inschrijving_status']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=True, stratify=y)

Oversampling the minority class

In [12]:
# from imblearn.over_sampling import RandomOverSampler, SMOTE

# # Using RandomOverSampler
# ros = RandomOverSampler(sampling_strategy='minority', random_state=42)
# X_train, y_train = ros.fit_resample(X_train, y_train)

# # Using SMOTE
# smote = SMOTE(sampling_strategy='minority', random_state=42)
# X_train, y_train = smote.fit_resample(X_train, y_train)

## Select and Train a Model

#### Soort Model:

- Recommender system 
- Model marketing pressure bepalen:
  - UNKNOWN

In [13]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble import VotingClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
# from sklearn.tree import DecisionTreeClassifier

# dec_clf = DecisionTreeClassifier(class_weight = {0: 0.989, 1: 1.045},random_state=42)
# rnd_clf = RandomForestClassifier(class_weight = {0: 0.989, 1: 1.045},n_estimators=25, n_jobs=10,random_state=42)
# svm_clf = SVC(class_weight = {0: 0.989, 1: 1.045},gamma="scale", random_state=42)

# voting_clf = VotingClassifier(
#     estimators=[('dt', dec_clf),('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
#     voting='hard', n_jobs=10)

In [14]:
# from sklearn.metrics import accuracy_score

# for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
#     clf.fit(X_train, y_train)
#     y_pred = clf.predict(X_test)
#     print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

In [15]:
# voting_clf.fit(X_train, y_train)


In [16]:
# print(voting_clf.score(X_test, y_test))

In [17]:
# from sklearn.ensemble import BaggingClassifier
# from sklearn.tree import DecisionTreeClassifier

# bag_clf = BaggingClassifier(
#     DecisionTreeClassifier(class_weight = {0: 0.989, 1: 1.045},random_state=42), n_estimators=25,
#      bootstrap=True) 
# bag_clf.fit(X_train, y_train)
# y_pred_bag = bag_clf.predict(X_test)

In [18]:
# from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score

# y_pred_bag = bag_clf.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred_bag)
# precision = precision_score(y_test, y_pred_bag)
# recall = recall_score(y_test, y_pred_bag)
# f1 = f1_score(y_test, y_pred_bag)
# roc_auc = roc_auc_score(y_test, y_pred_bag)

# print("Accuracy:", accuracy)
# print("Precision:", precision)
# print("Recall:", recall)
# print("F1 Score:", f1)
# print("ROC AUC Score:", roc_auc)

### Random Forest

In [24]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(class_weight = {0: 0.989, 1: 1.049},n_estimators=25, n_jobs=10,random_state=42)

In [25]:
rfc.fit(X_train, y_train)

RandomForestClassifier(class_weight={0: 0.989, 1: 1.049}, n_estimators=25,
                       n_jobs=10, random_state=42)

In [26]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score

y_pred_rfc = rfc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_rfc)
precision = precision_score(y_test, y_pred_rfc)
recall = recall_score(y_test, y_pred_rfc)
f1 = f1_score(y_test, y_pred_rfc)
roc_auc = roc_auc_score(y_test, y_pred_rfc)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC AUC Score:", roc_auc)

Accuracy: 0.9850041097723378
Precision: 0.9556459299739467
Recall: 0.39705882352941174
F1 Score: 0.5610206479771697
ROC AUC Score: 0.6983015386371505


In [27]:
from sklearn.metrics import confusion_matrix
print(X_test.shape)
print("*"*50)
print(sum(y_test==True))
print("*"*50)
matrix = confusion_matrix(y_test, y_pred_rfc)
print(matrix.diagonal()/matrix.sum(axis=1))
print("*"*50)
print(matrix)

(1569430, 9)
**************************************************
37876
**************************************************
[0.99954425 0.39705882]
**************************************************
[[1530856     698]
 [  22837   15039]]


In [28]:
throw KeyError

SyntaxError: invalid syntax (3018925666.py, line 1)

Grid Search CV, Boosting, Stacking

In [ ]:
from sklearn.model_selection import GridSearchCV

param = {'class_weight': [[{0:x, 1:w} for w in [1.01, 1.02, 1.03]for x in [0.5,0.75,0.8]]]}


grid = GridSearchCV(RandomForestClassifier(n_estimators=25, random_state=42), param, refit=True, verbose=True, n_jobs=10)


In [ ]:
grid.fit(X_train, y_train)
print(grid.best_params_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
{'class_weight': {0: 1, 1: 1}, 'n_estimators': 25}


In [ ]:
from sklearn.metrics import classification_report
grid_predictions = grid.predict(X_test)
print(classification_report(y_test, grid_predictions))

/home/flor/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/flor/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98    683109
         1.0       0.00      0.00      0.00     21462

    accuracy                           0.97    704571
   macro avg       0.48      0.50      0.49    704571
weighted avg       0.94      0.97      0.95    704571



/home/flor/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, grid_predictions)

array([[683109,      0],
       [ 21462,      0]])

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5, random_state=42)
ada_clf.fit(X_train, y_train)

### Neural network ?